In [155]:
import os
import pandas as pd

In [156]:
#create filepath
filepath1 = os.path.join("purchase_data.json")
filepath2 = os.path.join("purchase_data2.json")

In [157]:
#read the files into data-frames
data1_df = pd.read_json(filepath1)
data2_df = pd.read_json(filepath2)

In [158]:
#merge data1 & data2
lst = [data1_df, data2_df]
data_df = pd.concat(lst)
data_df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [159]:
#total_players = len(data_df["SN"])
total_players = data_df["SN"].nunique()
total_players

612

In [160]:
uniq_items = data_df["Item Name"].nunique()

In [161]:
avg_purchase_price = data_df["Price"].mean()

In [162]:
tot_purchases = len(data_df["Item Name"])

In [163]:
tot_rev = data_df["Price"].sum()

In [164]:
#create a data frame from the above calculations
purchasing_analysis = pd.DataFrame({"Number of Unique Items" : [uniq_items],
                                 "Average Purchase Price" : [avg_purchase_price],
                                 "Total Number of Purchases": [tot_purchases],
                                 "Total Revenue" : [tot_rev]
                                })
purchasing_analysis

,Average Purchase Price,Number of Unique Items,Total Number of Purchases,Total Revenue
0,2.930571,180,858,2514.43


In [165]:
#create a dataframe with no duplicate entries for buyers
players_df = data_df.drop_duplicates("SN",keep='first', inplace=False)

In [166]:
#groupby gender to extract gender related data
pd.DataFrame(players_df.groupby("Gender")["Gender"].count())

,Gender
Gender,
Female,108
Male,495
Other / Non-Disclosed,9


In [167]:
prcnt_female = (len(players_df.groupby("Gender").groups["Female"])/total_players)*100
prcnt_male = (len(players_df.groupby("Gender").groups["Male"])/total_players)*100
prcnt_other = (len(players_df.groupby("Gender").groups["Other / Non-Disclosed"])/total_players)*100

In [168]:
#create a data frame to summarize gender %
gender_analysis = pd.DataFrame({"% Male " : [prcnt_male],
                                "% Female": [prcnt_female],
                                "% other": [prcnt_other]
                                })
gender_analysis

,% Female,% Male,% other
0,17.647059,80.882353,1.470588


In [169]:
data_df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [173]:
# Use groupby & agg to perform purchase analysis on gender
avg_price = data_df.groupby("Gender",as_index=False).agg({"Price":"mean"})
avg_price = avg_price.rename(columns = {"Price":"average price"})
tot_price = data_df.groupby("Gender",as_index=False).agg({"Price":"sum"})
tot_price = tot_price.rename(columns = {"Price":"total purchase"})
tot_count = data_df.groupby("Gender",as_index=False).agg({"Price":"count"})
tot_count = tot_count.rename(columns = {"Price":"purchase count"})


,Gender,average price,total purchase,purchase count,Normalized totals
0,Female,2.847584,424.29,149,2.847584
1,Male,2.944448,2052.28,697,2.944448
2,Other / Non-Disclosed,3.155000,37.86,12,3.155000


In [ ]:
#Merge the above data frames to create a summary df
lst = [tot_count, avg_price, tot_price]
gender_df = pd.merge(avg_price, tot_price)
gender_df = pd.merge(gender_df,tot_count )
gender_df["Normalized totals"] = gender_df["total purchase"] / gender_df["purchase count"]
gender_df